# Personal Plotly API tests

In this notebook, I am learning plotly API!

## Imports and setup

We import plotly.offline and plotly.graph_objs.ob.

In [74]:
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

To plot figure call:

In [75]:
#fig = go.Figure(data,layout)
#iplot(fig)

### Data setup

Example data setup:

In [76]:
trace1 = dict(go.Scatter3d(
        x=[1,2,5,9],
        y=[5,0,8,2],
        z=[1,2,0,-1],
        mode='line',
        name='bubbles!',
        marker=dict(size=15),
        line = dict(width=2,color='rgb(21,117,22)')
    ))
data = [trace1]

### Layout setup

In [77]:
layout = dict(
    width=600,height=600,
    showlegend=True,
    title='2D Example',
    font = dict(family="Verdana"),
    scene = dict(
        xaxis = dict(range=[-1, 10], autorange=False, zeroline=False),
        yaxis = dict(range=[-1, 10], autorange=False, zeroline=False),
        aspectmode = 'cube',
        camera = dict(center=dict(x=0,y=0,z=0),eye=dict(x=1,y=-1,z=1))
    ),
    plot_bgcolor='rgb(255, 255, 255)'
)

In [78]:
fig=go.Figure(data=data,layout=layout)
iplot(fig)

# Making a plane

In [79]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
#numpy for calculations
import numpy as np

In [80]:
"""Create 2d mesh in sepecifies x and y axes limits, with number of points for every dimension separate."""
def mesh2d(xlim, ylim, n=5):
    if isinstance(n, int):
        xx = np.linspace(xlim[0],xlim[1],n)
        yy = np.linspace(ylim[0],ylim[1],n)
    elif isinstance(n, list):
        xx = np.linspace(xlim[0],xlim[1],n[0])
        yy = np.linspace(ylim[0],ylim[1],n[1])
    else:
        raise Exception("Wrong number of points parameter")
    return np.meshgrid(xx, yy)

In [81]:
"""Line class for intersections simulation
@author Nick
@since 25.07.17"""
class Line:
    
    def __init__(self, vec, offset):
        self.vec = vec
        self.offset = offset
        t = np.linspace(-1,1,3) #the parameter
        self.x = self.offset[0]+t*self.vec[0] #generate xpoints
        self.y = self.offset[1]+t*self.vec[1] #for y points
        self.z = self.offset[2]+t*self.vec[2] #for z points
    
    """Export the line into graphics object"""
    def go(self):
        line = go.Scatter3d(
            name="line",
            mode="lines",
            x=self.x,
            y=self.y,
            z=self.z,
            line = dict(
                color = ('rgb(205, 12, 24)'),
                width = 10)
            
        )
        return line

In [119]:
"""Plane class for intersections simulation
@author Nick
@since 25.07.17"""
class Plane:
    
    def __init__(self, normal, offset, xlim=[-1,1], ylim=[-1,1], zlim=[-1,1], n=10):
        self.normal = normal
        self.offset = offset
        if self.normal[2] == 0: #check if z is zero, then we have to generate x or y from other meshes
            if self.normal[1] == 0: #check if z and y is zero, then we have to generate x from yz mesh
                if self.normal[0] == 0: 
                    return ValueError("Normal vector is zero vector.")
                else:
                    #cannot generate z but can y, try generating y for xz mesh
                    self.y, self.z = mesh2d(ylim, zlim, n)
                    self.x = (np.dot(self.normal,self.offset)-normal[1]*self.y-normal[2]*self.z)/self.normal[0]
            else:
                #cannot generate z but can y, try generating y for xz mesh
                #self.normal[2] = 0.01 # TODO THIS IS VERY CRUDE
                self.x, self.z = mesh2d(xlim, zlim, n)
                self.y = (np.dot(self.normal,self.offset)-normal[0]*self.x-normal[2]*self.z)/self.normal[1]
        else:
            #try generating z
            self.x, self.y = mesh2d(xlim, ylim, n)
            #Generate plane z-values array
            self.z = (np.dot(self.normal,self.offset)-normal[0]*self.x-normal[1]*self.y)/self.normal[2]
    
    """Compute an intersection between this plane and 'other' plane. Returns Line object."""
    def intersection(self, other):
        if isinstance(other,Plane):
            #cross-product
            cross = np.cross(self.normal,other.normal)
            if np.all(cross==0):
                #planes are parallel or overlap
                raise ArithmeticError("Planes are parallel.")
            else:
                #sample point: x=0
                mat = [[self.normal[1],self.normal[2]],[other.normal[1], other.normal[2]]]
                axis = 0 #keep track of which axis we zero out
                #NOTE: this can raise np.linalg.LinAlgError exception when x=0 results in such a matrix
                #we have to check other cases
                if np.linalg.matrix_rank(mat) == 1:
                    mat = [[self.normal[0],self.normal[2]],[other.normal[0], other.normal[2]]]
                    axis = 1
                    if np.linalg.matrix_rank(mat) == 1:
                        mat = [[self.normal[0],self.normal[1]],[other.normal[0], other.normal[1]]]
                        axis = 2
                            
                rhs = [np.dot(self.normal,self.offset),np.dot(other.normal,other.offset)]
                sol = np.linalg.solve(mat,rhs)
                if axis == 0:
                    return Line(cross, [0,sol[0],sol[1]])
                if axis == 1:
                    return Line(cross, [sol[0],0,sol[1]])
                if axis == 2:
                    return Line(cross, [sol[0],sol[1],0])
        else:
            raise TypeError("'other' parameter is not a plane.")
    
    """Export the plane into graphics object"""
    def go(self):
        surf = go.Surface(
            name="plane",
            x=self.x,
            y=self.y,
            z=self.z)
        return surf
    
# equation of a plane: normal . (r - offset) = 0
plane1 = Plane([1.0,1.0,0.0],[0.0,0.0,0.0])
plane2 = Plane([1.0,1.0,-1.0],[0.0,1.0,1.0])
intersection = plane1.intersection(plane2)
print(intersection.offset,intersection.vec)

[0, 0.0, -0.0] [-1.  1.  0.]


In [120]:
#generate traces
trace1 = plane1.go()
trace2 = plane2.go()
trace3 = plane1.intersection(plane2).go()
data = [trace1,trace2,trace3]

In [121]:
#define layout
layout = dict(
    width=600,height=600,
    showlegend=True,
    title='Plane intersection example',
    font = dict(family="Verdana"),
    scene = dict(
        xaxis = dict(range=[-1, 1], autorange=False, zeroline=False),
        yaxis = dict(range=[-1, 1], autorange=False, zeroline=False),
        zaxis = dict(range=[-1, 1], autorange=False, zeroline=False),
        aspectmode = 'cube',
        camera = dict(center=dict(x=0,y=0,z=0),eye=dict(x=1,y=-1,z=1))
    ),
    plot_bgcolor='rgb(255, 255, 255)'
)

In [122]:
#plot the figure
fig=go.Figure(data=data,layout=layout)
iplot(fig)